# Age-Abundance Relation

In [2]:
from matplotlib import pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd
import yaml

In [3]:
from auriga.snapshot import Snapshot
from auriga.images import figure_setup
from auriga.settings import Settings
from auriga.parser import parse
from auriga.support import make_snapshot_number

In [4]:
figure_setup()
settings = Settings()

In [5]:
CONFIG_FILE = "02"
config = yaml.safe_load(open(f"../configs/{CONFIG_FILE}.yml"))

In [6]:
SAMPLE = [f"au{i}_or_l4_s127" for i in settings.groups["Included"]]

In [8]:
def read_data(simulation: str, of: str, to: str) -> pd.DataFrame:
    s = Snapshot(simulation=simulation, loadonlytype=[0, 1, 2, 3, 4, 5])
    s.add_stellar_age()
    s.add_metal_abundance(of, to)
    s.add_circularity()
    s.tag_particles_by_region(
        disc_std_circ=config["DISC_STD_CIRC"],
        disc_min_circ=config["DISC_MIN_CIRC"],
        cold_disc_delta_circ=config["COLD_DISC_DELTA_CIRC"],
        bulge_max_specific_energy=config["BULGE_MAX_SPECIFIC_ENERGY"])

    is_real_star = (s.type == 4) & (s.stellar_formation_time > 0)
    is_main_obj = (s.halo == s.halo_idx) & (s.subhalo == s.subhalo_idx)

    props = {
        "StellarAge_Gyr": s.stellar_age[is_real_star & is_main_obj],
        f"[{of}/{to}]": s.metal_abundance[f"{of}/{to}"][
            is_real_star & is_main_obj],
        "ComponentTag": s.region_tag[is_real_star & is_main_obj]}

    df = pd.DataFrame(props)
    df[~np.isfinite(df)] = np.nan
    df.dropna(inplace=True)
    return df

In [70]:
of, to = "O", "H"

# Calculate the medians for all galaxies in the sample
data = {"Simulation": []}
for component in settings.components:
    data[f"Abundance_{of}{to}_{component}"] = []
    data[f"StellarAge_{component}_Gyr"] = []

for simulation in SAMPLE:
    data["Simulation"].append(simulation)
    df = read_data(simulation=simulation, of=of, to=to)
    for i in range(len(settings.components)):
        component = settings.components[i]
        is_region = (df["ComponentTag"] == i)
        data[f"Abundance_{of}{to}_{component}"].append(
            np.nanmedian(df[f"[{of}/{to}]"][is_region])
        )
        data[f"StellarAge_{component}_Gyr"].append(
            np.nanmedian(df[f"StellarAge_Gyr"][is_region])
        )

df = pd.DataFrame(data)
df.to_csv(
    f"../results/age_{of}{to}_abundance_sample_"
    f"median{config['FILE_SUFFIX']}.csv")

In [69]:
markers = list(settings.component_markers.values())
colors = list(settings.component_colors.values())
labels = list(settings.component_labels.values())

fig, axs = plt.subplots(
    figsize=(7.4, 2.0), nrows=1, ncols=4, sharey=True, sharex=True,
    gridspec_kw={"hspace": 0.0, "wspace": 0.0})

for ax in axs.flat:
    ax.grid(True, ls='-', lw=0.25, c="gainsboro")
    ax.tick_params(which='both', direction="in")

    ax.set_xlim(0, 14)
    ax.set_xticks([2, 4, 6, 8, 10, 12])
    ax.set_xlabel("median(Age) [Gyr]")

    ax.set_ylim(-0.7, 0.9)
    ax.set_ylabel(f"median([{of}/{to}])")
    ax.set_axisbelow(True)

    ax.label_outer()

for i, ax in enumerate(axs.flatten()):
    for j in range(4):
        component = settings.components[j]
        label = labels[j] if j == i else None
        color = colors[j] if j == i else "silver"
        zorder = 10 if j == i else 5
        ax.scatter(
            df[f"StellarAge_{component}_Gyr"],
            df[f"Abundance_{of}{to}_{component}"],
            c=color, label=label, zorder=zorder,
            s=20, linewidths=0.4, edgecolors="white", marker=markers[j])

        if j == i:
            age_pdf = gaussian_kde(df[f"StellarAge_{component}_Gyr"])
            pdf_x = np.linspace(0, 14, 100)
            pdf_y = age_pdf(pdf_x)
            pdf_y = (pdf_y - pdf_y.min()) / (pdf_y.max() - pdf_y.min()) * 0.1 * np.diff(ax.get_ylim()) + ax.get_ylim()[0]
            ax.fill_between(
                x=pdf_x, y1=ax.get_ylim()[0], y2=pdf_y, edgecolor=color,
                facecolor=mcolors.TABLEAU_COLORS[color] + "30")
            
            abundance_pdf = gaussian_kde(df[f"Abundance_{of}{to}_{component}"])
            pdf_y = np.linspace(-1.0, 0.7, 100)
            pdf_x = abundance_pdf(pdf_y) / np.max(abundance_pdf(pdf_y)) * 2
            ax.fill_betweenx(
                y=pdf_y, x1=-pdf_x + 14, x2=14, edgecolor=color,
                facecolor=mcolors.TABLEAU_COLORS[color] + "30")

    ax.legend(loc="upper left", framealpha=0, fontsize=7.5)

fig.savefig(
    f"../images/age_{of}{to}_abundance_by_region/"
    f"included_scatter{config['FILE_SUFFIX']}.pdf")
plt.close(fig)